Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = ""
COLLABORATORS = ""

---

# CSE 30 Spring 2022 - Bonus Homework!

Copyright Luca de Alfaro, 2021.  License: [CC-BY-NC-ND](https://creativecommons.org/licenses/by-nc-nd/4.0/).

### Instructions

Please disregard the YOUR NAME and COLLABORATORS above.  They are put there atomatically by the grading tool.
You can find instructions on how to work on a homework on Canvas.  Here is a short summary: 

### Submitting your work

To submit your work: 

* First, click on "Runtime > Restart and run all", and check that you get no errors.  This enables you to catch any error you might have introduced, and not noticed, due to your running cells out of order. 
* Second, download the notebook in .ipynb format (File > Download .ipynb) and upload the .ipynb file to [this form](https://docs.google.com/forms/d/e/1FAIpQLSc0GKeAJ0e4dVHQ3Wn6gS8hftafYr7g5PohmQ4LJWXo9Xo0XQ/viewform?usp=sf_link).  This homework is due at **11:59pm on Sunday, 05 June 2022**, but it's completely optional.

You can submit multiple times; the last submission before the deadline is the one that counts.

### Homework format

For each question in this notebook, there is: 

* A text description of the problem. 
* One or more places where you have to insert your solution.  You need to complete every place marked: 

    `# YOUR CODE HERE`
    
    and you should not modify any other place. 
* One or more test cells.  Each cell is worth some number of points, marked at the top.  You should not modify these tests cells.  The tests pass if no error is printed out: when there is a statement that says, for instance: 

    `assert x == 2`
    
    then the test passes if `x` has value 2, and fails otherwise.  You can insert a `print(x)` (for this case!) somewhere if you want to debug your work; it is up to you.  
    
### Notes:

* Your code will be tested both according to the tests you can see (the `assert` statements you can see), _and_ additional tests.  This prevents you from hard-coding the answer to the particular questions posed.  Your code should solve the _general_ intended case, not hard-code the particular answer for the values used in the tests. 

* **Please do not delete or add cells!** The test is autograded, and if you modify the test by adding or deleting cells, even if you re-add cells you delete, you may not receive credit. 

* **Please do not import modules that are not part of the [standard library](https://docs.python.org/3/library/index.html).** You do not need any, and they will likely not available in the grading environment, leading your code to fail. 

* **If you are inactive too long, your notebook might get disconnected from the back-end.** Your work is never lost, but you have to re-run all the cells before you continue. 

* You can write out print statements in your code, to help you test/debug it. But remember: the code is graded on the basis of what it outputs or returns, not on the basis of what it prints.

* **TAs and tutors have access to this notebook,** so if you let them know you need their help, they can look at your work and give you advice. 

### Grading

Each cell where there are tests is worth a certain number of points.  You get the points allocated to a cell only if you pass _all_ the tests in the cell. 

The tests in a cell include both the tests you can see, and other, similar, tests that are used for grading only.  Therefore, you cannot hard-code the solutions: you really have to solve the essence of the problem, to receive the points in a cell. 

### Code of Conduct

* Work on the test yourself, alone. 
* You can search documentation on the web, on sites such as the Python documentation sites, Stackoverflow, and similar, and you can use the results. 
* You cannot share your work with others or solicit their help.


# The Holy Queens Problem
Luca de Alfaro, 2021

A chess queen can strike along the row, column, and the two diagonals.   The classical $n$-queens problem consists in placing $n$ queens on the chess board, so that no queen can eat another queen. 

We will study here the _holy_ queen problem: the chessboard can have _holes_, and in fact, can have an arbitrary shape.  The ability of a queen to move and attack is limited by the holes, and in general by the borders of the board: a queen cannot "jump" across a hole.  Your goal is to check whether you can place $n$ queens on a given board, which may contain holes. 

## The board

For a board, we will use a Numpy representation as a matrix, with the following conventions for the content of each square: 

* 0: the cell is available for a queen. 
* 1: a queen is in the cell.
* 2: the cell is under attack from some queen (and thus not available). 
* 3: the cell contains a wall/hole, and a queen cannot traverse it.  

We provide for you here a function `show_board` that visualizes a board, using a `Q` for a queen, a dot for an empty location, a `#` for a hole or wall, and a `*` for a cell under attack.  

In [2]:
QUEEN = 1
EMPTY = 0
FORBIDDEN = 2
WALL = 3

def show_board(board):
    rows, cols = board.shape
    for r in range(rows):
        s = ""
        for c in range(cols):
            if board[r, c] == QUEEN:
                s += "Q"
            elif board[r, c] == FORBIDDEN:
                s += "*"
            elif board[r, c] == WALL:
                s += "#"
            elif board[r, c] == EMPTY:
                s += "."
            else:
                s += "?"
        print(s)


In [3]:
import numpy as np

board=np.array([
                [0, 0, 0, 0, 0, 0],
                [0, 1, 0, 3, 3, 0],
                [0, 0, 0, 3, 3, 0],
                [0, 0, 1, 0, 0, 0],
                [0, 0, 0, 0, 0, 0]
])
show_board(board)


......
.Q.##.
...##.
..Q...
......


Let us also write the opposite function `read_board`. 

In [4]:
def read_board(string_list):
    rows = len(string_list)
    cols = len(string_list[0])
    board = np.zeros((rows, cols))
    for r, row in enumerate(string_list):
        assert len(row) == cols
        for c, s in enumerate(row):
            if s == "Q":
                board[r, c] = QUEEN
            elif s == "#":
                board[r, c] = WALL
            elif s == "*":
                board[r, c] = FORBIDDEN
    return board


In [5]:
bs = [
    "......",
    ".Q.##.",
    "...##.",
    "..Q...",
    "......"
    ]
show_board(read_board(bs))


......
.Q.##.
...##.
..Q...
......


Here is the class `HolyQueens`.  You must define two methods: 

The method `propagate` propagates the constraints, marking with `FORBIDDEN` all the locations that can be reached by the queens on the board. 

The method `search` searches for a solution with a given number of queens.  If a solution is found, it returns the board.  If no solution is found, it returns `None`.  The method `search` should implement the propagate-guess-recur framework: if not enough queens are present on the board, it first propagates the constraints from the current queens if any, then it guesses the position for a new queen, and then it recurs, looking for a solution in which one fewer queen needs to be placed. 

We advise you to implement `propagate` first, and then `search`. 

In [6]:
import random
class HolyQueens(object):

    def __init__(self, board):
        self.board = board
        self.num_rows, self.num_cols = self.board.shape
        # Current number of queens in the board.
        self.num_queens = np.sum(self.board == QUEEN)

    def show(self):
        show_board(self.board)

    def propagate(self):
        """Propagates the information on the board, marking with 2 the
        positions where a queen cannot be."""
        # The solution can be written concisely in about 20 lines of code,
        # but if you brute force it, it might be quite long.
        # YOUR CODE HERE
        queen_location = list(zip(*np.where(self.board == QUEEN)))
        for q in queen_location:
           directions = [(-1, 0), (1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)]
           for dir in directions:
              temp = q
              while True:
                temp = tuple(np.add(temp, dir))
                if temp[0] < 0 or temp[0] >= self.num_rows or temp[1] < 0 or temp[1] >= self.num_cols or self.board[temp[0]][temp[1]] == WALL:
                  break
                else:
                  self.board[temp[0]][temp[1]] = FORBIDDEN

    def search(self, total_num_queens):
        """Searches for a solution, starting from the given board,
        which contains exactly num_queens.  It returns the board,
        if such a solution is found, or None, if none could be found
        after exhaustive search."""
        pass
        # YOUR CODE HERE
        try:
          if len(list(zip(*np.where(self.board == QUEEN)))) == total_num_queens:
            return self.board
          if len(np.array(np.where(self.board==EMPTY)).T) == 0:
            self.board[self.board == QUEEN] = EMPTY
            self.board[self.board == FORBIDDEN] = EMPTY
            return self.search(total_num_queens)
          able_to_occupy = list(zip(*np.where(self.board == EMPTY)))
          square_r = random.choice(able_to_occupy)
          self.board[square_r[0]][square_r[1]] = QUEEN
          self.propagate()
          return self.search(total_num_queens)
        except RecursionError as re:
          return None



Let us see how propagate works. 

In [7]:
# Propagating this
bs = [
    "......",
    ".Q.##.",
    "...##.",
    "..Q...",
    "......"
    ]
# should give this:
bs_prop = [
    "***...",
    "*Q*##.",
    "***##.",
    "**Q***",
    ".****."]

hq = HolyQueens(read_board(bs))
hq.propagate()
hq.show()
assert (hq.board == read_board(bs_prop)).all()


***...
*Q*##.
***##.
**Q***
.****.


In [8]:
# Propagating this
bs = [
    ".....Q",
    "..Q##.",
    "...##.",
    ".#....",
    ".Q...."
    ]
# should give this:
bs_prop = [
    "*****Q",
    "**Q##*",
    ".**##*",
    "*#*..*",
    "*Q****"]

hq = HolyQueens(read_board(bs))
hq.propagate()
hq.show()
assert (hq.board == read_board(bs_prop)).all()


*****Q
**Q##*
.**##*
*#*..*
*Q****


In [9]:
## Here you can write more tests.


Let us check the search function now. 

In [10]:
bs = [
    "......",
    "...##.",
    "...##.",
    "......",
    "......"
    ]
hq = HolyQueens(read_board(bs))
r = hq.search(4)
assert r is not None
# You should get a solution with 4 non-interfering queens.
show_board(r)


****.*
**Q##*
Q**##Q
***.**
*Q****


In [11]:
bs = [
    "......",
    "...##.",
    "...##.",
    "......",
    "......"
    ]
hq = HolyQueens(read_board(bs))
r = hq.search(6)
assert r is not None
# You should get a solution with 6 non-interfering queens.
show_board(r)


***Q**
Q**##Q
**Q##*
****Q*
*Q****
